# Data Analyst Jobs

Data analysis is a process of inspecting, cleansing, transforming and modeling data with the goal of discovering useful information, informing conclusions and supporting decision-making. Data analysis has multiple facets and approaches, encompassing diverse techniques under a variety of names, and is used in different business, science, and social science domains. In today's business world, data analysis plays a role in making decisions more scientific and helping businesses operate more effectively. Data mining is a particular data analysis technique that focuses on statistical modeling and knowledge discovery for predictive rather than purely descriptive purposes, while business intelligence covers data analysis that relies heavily on aggregation, focusing mainly on business information. In statistical applications, data analysis can be divided into descriptive statistics, exploratory data analysis, and confirmatory data analysis.

## Here we'll be performing:

* Data Cleaning
* Data Visualization

# Importing Essentials

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Meeting the Data

In [ ]:
df=pd.read_csv('/kaggle/input/data-analyst-jobs/DataAnalyst.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

# Data Cleaning

Many of the columns contain -1 values. Let's replace it with nan

In [ ]:
df=df.replace(-1.0,np.NaN)
df=df.replace(-1,np.NaN)
df=df.replace('-1',np.NaN)

In [ ]:
df.isnull().sum()

#### Lot's of null values!

Dropping uwanted columns

In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)

Splitting ' , ' from "Job Title"

In [ ]:
df['Job Title'],df['Department']=df['Job Title'].str.split(', ',1).str

Splitting "\n" from Company Name

In [ ]:
df['Company Name'],_=df['Company Name'].str.split('\n',1).str

Splitting Location into states and cities

In [ ]:
df['City'],df['State']=df['Location'].str.split(', ',1).str
df.drop('Location',axis=1,inplace=True)

Filling the null values in easy apply as false(No Vaccancy)

In [ ]:
df['Easy Apply']=df['Easy Apply'].fillna(False).astype('bool')

Dividing the salary range

In [ ]:
# Remove the the texts within brackets
df['Salary Estimate'],_=df['Salary Estimate'].str.split('(',1).str

#Replacing K with ""
df['Salary Estimate']=df['Salary Estimate'].replace('K','',regex=True)

# Creating two columns Maximum and Minimum salaries by splitting the "-" 
df['Minimum Salary'],df['Maximum Salary']=df['Salary Estimate'].str.split('-',1).str

# Removing the $ sign
df['Minimum Salary']=df['Minimum Salary'].str.lstrip('$').fillna(0).astype('int')
df['Maximum Salary']=df['Maximum Salary'].str.lstrip('$').fillna(0).astype('int')

# Dropping the salary estimate column
df.drop('Salary Estimate',axis=1,inplace=True)

# Data Analysis and Visualization

In [ ]:
df['Revenue'].value_counts()

## What's the maximum and minimum salary distribution?

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(15,5))
_=sns.distplot(df['Minimum Salary'],ax=ax[0])
_=sns.distplot(df['Maximum Salary'],ax=ax[1])

## Distribution of Rating

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(df['Rating']);

## Top 10 Job Titles

In [ ]:
df['Job Title'].value_counts().head(10).plot.bar();

## Lowest paid jobs

In [ ]:
df_min=df.groupby(['Minimum Salary'])[['Job Title','Minimum Salary']]
df_min.head().nsmallest(10,'Minimum Salary')

## Highest paid Jobs

In [ ]:
df_max=df.groupby(['Maximum Salary'])[['Job Title','Maximum Salary']]
df_max.head().nlargest(10,'Maximum Salary')

## 50 Companies with vaccancies and job titles

In [ ]:
data=df[['Job Title','Company Name']]
data[data['Easy Apply']==True].sort_values(by='Easy Apply',ascending=False).head(50)

## Top 10 Companies

In [ ]:
df['Company Name'].value_counts().head(10).plot.bar();

## Top 10 highest Rated Companies

In [ ]:
df_rating=df.groupby(['Rating'])['Company Name','Type of ownership','Rating']
df_rating.head().nlargest(10,'Rating')

## 10 highest rated companies with vaccancies

In [ ]:
df[df['Easy Apply']==True][['Company Name','Easy Apply','Rating']].head(10).nlargest(10,'Rating')

## Top 10 Headquaters

In [ ]:
df['Headquarters'].value_counts().head(10).plot.bar();

In [ ]:
plt.figure(figsize=(8,8))
df['Headquarters'].value_counts().head(10).plot.pie(autopct='%.2f%%')
plt.axis('off');

## Top 10 Companies founded in a year

In [ ]:
df['Founded'].value_counts().head(10).plot.bar();

In [ ]:
plt.figure(figsize=(12,6))
df['Founded'].value_counts().head(10).plot.pie(autopct='%.2f%%')
plt.axis('off');

## Top 10 Industries

In [ ]:
plt.figure(figsize=(8,5))
df['Industry'].value_counts().head(10).plot.bar();

In [ ]:
plt.figure(figsize=(8,8))
df['Industry'].value_counts().head(10).plot.pie(autopct='%.2f%%')
plt.axis('off');

## Top 10 Sectors

In [ ]:
plt.figure(figsize=(8,5))
df['Sector'].value_counts().head(10).plot.bar();

In [ ]:
plt.figure(figsize=(8,8))
df['Sector'].value_counts().head(10).plot.pie(autopct='%.2f%%')
plt.axis('off');

## Top 10 Ownerships

In [ ]:
plt.figure(figsize=(8,5))
df['Type of ownership'].value_counts().head(10).plot.bar();

In [ ]:
plt.figure(figsize=(9,9))
df['Type of ownership'].value_counts().head(10).plot.pie(autopct='%.2f%%')
plt.axis('off');

# WordCloud (Job Title)

In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'black',
                     height=1500,
                     width=1500).generate(str(df["Job Title"]))
plt.rcParams['figure.figsize'] = (12,12)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Most available Job Title")
plt.show()

# WordCloud (Company Name)

In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'black',
                     height=1500,
                     width=1500).generate(str(df["Company Name"]))
plt.rcParams['figure.figsize'] = (12,12)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Most available Companies")
plt.show()

# Like my work?

### Please do upvote